In [ ]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install sacrebleu
!pip install rouge_score
!pip install py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt

import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

 HERE BELOW I JUST WANT TO IMPORT THE DATASET FROM DATASETS LIBRARY AND ALSO I WANT TO CHECK THE COLUMNS IN THE DATASET.



In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", version="3.0.0")

print( f"{dataset['train'].column_names}")

  0%|          | 0/3 [00:00<?, ?it/s]

['article', 'highlights', 'id']


In [ ]:
example=dataset["train"][1]
print(f"""
FULL LENGTH {len(example["article"])}):
""")
print(f'Summary length: {len(example["highlights"])}):')


FULL LENGTH 4051):

Summary length: 281):


SO ABOVE WE CAN OBSERVE THAT THE SUMMARY HAS 281 WORDS AS COMPARED TO THE FULL LENGGTH OF THE FIRST ARTICLE WHICH CONATINS 4051 WORDS

In [ ]:
example_summary= dataset["train"][1]["article"][:1000]
summaries = {}

In [ ]:

def default_summary(text):
    return "\n".join(sent_tokenize(text)[:3])
summaries['baseline'] = default_summary(example_summary)

summaries['baseline']

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."'

In [ ]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(example_summary)
pipe_out

[{'summary_text': 'Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most severe mental illnesses are incarcerated. Most often, they face drug charges or charges of assaulting an officer. Judge Steven Leifman says the arrests often result from confrontations with police.'}]

In [ ]:
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
summaries["bart"]

'Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most severe mental illnesses are incarcerated.\nMost often, they face drug charges or charges of assaulting an officer.\nJudge Steven Leifman says the arrests often result from confrontations with police.'

In [ ]:
pipe = pipeline('summarization', model="google/pegasus-cnn_dailymail"  )

pipe_out = pipe(example_summary)
pipe_out

[{'summary_text': 'Mentally ill inmates are housed on the "forgotten floor" of a Miami jail .<n>Judge Steven Leifman says the charges are usually "avoidable felonies"<n>He says the arrests often result from confrontations with police .<n>Mentally ill people often won\'t do what they\'re told when police arrive on the scene .'}]

In [ ]:
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

since i have used 2 models which are **BART** and **pegasus** i need to compare them to see which model gives the better summarization compared to each other and which model is closer to the actual human summarization done by the CNN editors

**comparision**:

In [ ]:
print("REAL SUMMARY BY EDITORS --->")

print(dataset['train'][1]['highlights'])


for allmodels in summaries:
    print(allmodels.upper(),"--->")
    print(summaries[allmodels])

REAL SUMMARY BY EDITORS --->
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
BASELINE --->
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."
BART --->
Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most severe mental illnesses are incarcerated.
Most often, they face drug charges or charges of assa

As told in the assignment i will be using both bleu and rouge metrics to analyze my outputs from BART and pegasus and compare

In [ ]:
from datasets import load_metric

bm = load_metric("sacrebleu")
rm=load_metric("rouge")

<ipython-input-16-4704b00dbb31>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bm = load_metric("sacrebleu")


we can observe the bleu metric score for the pegasus model below

In [ ]:
bm.add(prediction = [summaries["pegasus"]], reference = [dataset['train'][1]['highlights'] ])

results = bm.compute(smooth_method = 'floor', smooth_value = 0 )
results

{'score': 18.738410352664832,
 'counts': [27, 14, 10, 6],
 'totals': [67, 66, 65, 64],
 'precisions': [40.298507462686565,
  21.21212121212121,
  15.384615384615385,
  9.375],
 'bp': 1.0,
 'sys_len': 67,
 'ref_len': 57}

we can observe the bleu metric score for the bart model below

In [ ]:
bm.add(prediction = [summaries["bart"]], reference = [dataset['train'][1]['highlights'] ])

results = bm.compute(smooth_method = 'floor', smooth_value = 0 )
results

{'score': 10.887081348477308,
 'counts': [19, 7, 5, 2],
 'totals': [55, 54, 53, 52],
 'precisions': [34.54545454545455,
  12.962962962962964,
  9.433962264150944,
  3.8461538461538463],
 'bp': 0.9642895789647244,
 'sys_len': 55,
 'ref_len': 57}

ROUGE METRIC SCORES for PEGASUS


In [ ]:
rm.add(prediction = [summaries["pegasus"]], reference = [dataset['train'][1]['highlights'] ])
results=rm.compute()
results

{'rouge1': AggregateScore(low=Score(precision=0.49019607843137253, recall=0.5102040816326531, fmeasure=0.5), mid=Score(precision=0.49019607843137253, recall=0.5102040816326531, fmeasure=0.5), high=Score(precision=0.49019607843137253, recall=0.5102040816326531, fmeasure=0.5)),
 'rouge2': AggregateScore(low=Score(precision=0.24, recall=0.25, fmeasure=0.24489795918367346), mid=Score(precision=0.24, recall=0.25, fmeasure=0.24489795918367346), high=Score(precision=0.24, recall=0.25, fmeasure=0.24489795918367346)),
 'rougeL': AggregateScore(low=Score(precision=0.35294117647058826, recall=0.3673469387755102, fmeasure=0.36000000000000004), mid=Score(precision=0.35294117647058826, recall=0.3673469387755102, fmeasure=0.36000000000000004), high=Score(precision=0.35294117647058826, recall=0.3673469387755102, fmeasure=0.36000000000000004)),
 'rougeLsum': AggregateScore(low=Score(precision=0.35294117647058826, recall=0.3673469387755102, fmeasure=0.36000000000000004), mid=Score(precision=0.3529411764

ROUGE METRIC SCORES for BART

In [ ]:
rm.add(prediction = [summaries["bart"]], reference = [dataset['train'][1]['highlights'] ])
results=rm.compute()
results

{'rouge1': AggregateScore(low=Score(precision=0.38636363636363635, recall=0.3469387755102041, fmeasure=0.3655913978494624), mid=Score(precision=0.38636363636363635, recall=0.3469387755102041, fmeasure=0.3655913978494624), high=Score(precision=0.38636363636363635, recall=0.3469387755102041, fmeasure=0.3655913978494624)),
 'rouge2': AggregateScore(low=Score(precision=0.13953488372093023, recall=0.125, fmeasure=0.13186813186813184), mid=Score(precision=0.13953488372093023, recall=0.125, fmeasure=0.13186813186813184), high=Score(precision=0.13953488372093023, recall=0.125, fmeasure=0.13186813186813184)),
 'rougeL': AggregateScore(low=Score(precision=0.20454545454545456, recall=0.1836734693877551, fmeasure=0.1935483870967742), mid=Score(precision=0.20454545454545456, recall=0.1836734693877551, fmeasure=0.1935483870967742), high=Score(precision=0.20454545454545456, recall=0.1836734693877551, fmeasure=0.1935483870967742)),
 'rougeLsum': AggregateScore(low=Score(precision=0.20454545454545456, 

from the above scores of the models we can observe that PEGASUS outperforms the bart model

In [ ]:
def calculate_metrics(dataset, metric, column_text = 'article', column_summary = 'highlights' ):

    summaries = [default_summary(text) for text in dataset[column_text] ]

    metric.add_batch(predictions = summaries, references = dataset[column_summary] )

    score = metric.compute()
    return score

In [ ]:
testing = dataset['train'].select(range(1000))
score = calculate_metrics(testing, rm )
score

{'rouge1': AggregateScore(low=Score(precision=0.2555911406253523, recall=0.42262922836959854, fmeasure=0.31238845832279244), mid=Score(precision=0.26109783598073155, recall=0.4325187490388759, fmeasure=0.31902933865554595), high=Score(precision=0.26715089646614104, recall=0.44338494329041706, fmeasure=0.3263087017667559)),
 'rouge2': AggregateScore(low=Score(precision=0.09760512302354517, recall=0.16479655661576284, fmeasure=0.12011634664758306), mid=Score(precision=0.1023288406329687, recall=0.17266206500643264, fmeasure=0.12590619245290882), high=Score(precision=0.10719323502500917, recall=0.18038456342950027, fmeasure=0.13151460560107112)),
 'rougeL': AggregateScore(low=Score(precision=0.16893217045182957, recall=0.2793656983457264, fmeasure=0.20629063201906134), mid=Score(precision=0.1736972892458265, recall=0.2883214434666669, fmeasure=0.2122187273930115), high=Score(precision=0.17849924849428844, recall=0.2971470706088962, fmeasure=0.21824366503719808)),
 'rougeLsum': AggregateSc